# Анна Гаврилина
## Лабораторная работа 3
### РЕШЕНИЕ СИСТЕМ ЛИНЕЙНЫХ АЛГЕБРАИЧЕСКИХ УРАВНЕНИЙ ПРЯМЫМИ МЕТОДАМИ. ТЕОРИЯ ВОЗМУЩЕНИЙ
## Вариант 6 : 3.1.6, 3.4, 3.10.2

## Задача 3.1 Дана система уравнений Ax=b порядка n. Исследовать зависимость погрешности решения x от погрешностей правой части системы b.

In [1]:
import numpy as np


Зададим матрицу системы A и вектор правой части b

In [103]:
n = 7
N = 6
A = np.array([[3 / ((0.1 * N * i * j)**4-4 * (0.1 * N *i*j)**3) for j in range (1, n + 1)] for i in range (1, n + 1) ])

In [104]:
b = np.ones(n) * N

In [11]:
print ("A: \n",A)
print ("b: \n",b)

A: 
 [[-4.08496732e+00 -6.20039683e-01 -2.33819678e-01 -1.35633681e-01
  -1.11111111e-01 -1.60751029e-01  2.02461937e-01]
 [-6.20039683e-01 -1.35633681e-01 -1.60751029e-01  3.39084201e-02
   6.94444444e-03  2.51173483e-03  1.15035192e-03]
 [-2.33819678e-01 -1.60751029e-01  1.36085527e-02  2.51173483e-03
   8.23045267e-04  3.50220106e-04  1.74385820e-04]
 [-1.35633681e-01  3.39084201e-02  2.51173483e-03  6.05507502e-04
   2.17013889e-04  9.66051856e-05  4.94291839e-05]
 [-1.11111111e-01  6.94444444e-03  8.23045267e-04  2.17013889e-04
   8.08080808e-05  3.67430923e-05  1.90552411e-05]
 [-1.60751029e-01  2.51173483e-03  3.50220106e-04  9.66051856e-05
   3.67430923e-05  1.69140392e-05  8.84267720e-06]
 [ 2.02461937e-01  1.15035192e-03  1.74385820e-04  4.94291839e-05
   1.90552411e-05  8.84267720e-06  4.64777983e-06]]
b: 
 [6. 6. 6. 6. 6. 6. 6.]


Найдем решение системы

In [7]:
x = np.linalg.solve(A, b)
print ('Решение системы Ax = b: \n', x)

Решение Ax = b: 
 [ 4.22830736e+00  8.63582825e+02  1.16140840e+05  5.36275970e+06
 -5.13172315e+07  6.46491798e+07  2.68968736e+07]


In [12]:
#число обусловленности матрицы
cond_A = np.linalg.cond(A)
print ('Число обусловленности матрицы A:', cond_A)

Число обусловленности матрицы A: 119587319.47430816


Принимая решение x, полученное в п. 1, за точное, вычислю вектор d = (d1,...,dn)T

In [15]:
eps = 0.001
d = np.zeros(n)

In [42]:
import copy

for i in range(n):
    b_k = copy.copy(b)
    b_k[i] += eps
    x_i = np.linalg.solve(A,b_k)
    d[i] = np.linalg.norm(x - x_i, np.inf) / np.linalg.norm(x, np.inf)

print ("Вектор относительных погрешностей решений: ")
print(d.reshape(7,1))  
print ("\nИндекс компоненты, оказывающая наибольшее влиянее на погрешность решения:", np.argmax(d))

Вектор относительных погрешностей решений: 
[[4.61719432e-11]
 [4.27730063e-09]
 [4.72993298e-07]
 [2.07123188e-05]
 [1.93242963e-04]
 [2.39878841e-04]
 [9.88413509e-05]]

Индекс компоненты, оказывающая наибольшее влиянее на погрешность решения: 5


Оценка теоретической погрешности

In [41]:
#теоретическая погрешность решения
cond_A * np.linalg.norm(b - b_k, np.inf) / np.linalg.norm(b, np.inf)

19931.21991239135

Практическая погрешность много меньше теоретической. Значение теоретической погрешности очень велико из-за того, что матрица А плохо обусловленна (число обусловненности велико)

## Задача 3.4 Решить систему уравнений Ax=b из задачи выше, используя LU-разложение матрицы A

In [43]:
from scipy.linalg import lu_factor

In [105]:
lu, piv = lu_factor(A)

Получу верхнюю и нижнюю треугольные матрицы

In [111]:
u = np.triu(lu)
l = np.tril(lu, k = -1) + np.eye(n)
#print("L:\n",l)
#print("U:\n",u)


Преобразаю вектор b по прямому ходу метода Гаусса

In [112]:
b_new = np.linalg.inv(l)@b
print(b_new)

[ 6.          5.65656566  3.21428571  8.7890625  12.30185097  6.25729536
  4.20920532]


Обратный ход

In [109]:
#решение нашей системы
x.reshape(7,1)

array([[ 4.22830736e+00],
       [ 8.63582825e+02],
       [ 1.16140840e+05],
       [ 5.36275970e+06],
       [-5.13172315e+07],
       [ 6.46491798e+07],
       [ 2.68968736e+07]])

In [113]:

x_new = []
x_new.append(b_new[-1]/u[-1][-1])
for i in range (n-2, -1, -1):
    x_m = b_new[i] - np.dot(u[i][i+1:],x_new[-1::-1])
    x_m /= u[i][i]
    x_new.append(x_m)

In [124]:
solitions = np.array(x_new)
print("Решение, полученное обратным ходом:\n", solutions.reshape(7,1))
print("\nРешение, полученнное в пункте 3.1:\n",x.reshape(7,1))

Решение, полученное обратным ходом:
 [[ 4.22830736e+00]
 [ 8.63582825e+02]
 [ 1.16140840e+05]
 [ 5.36275970e+06]
 [-5.13172315e+07]
 [ 6.46491798e+07]
 [ 2.68968736e+07]]

Решение, полученнное в пункте 3.1:
 [[ 4.22830736e+00]
 [ 8.63582825e+02]
 [ 1.16140840e+05]
 [ 5.36275970e+06]
 [-5.13172315e+07]
 [ 6.46491798e+07]
 [ 2.68968736e+07]]


## Задача 3.10.2 Дана система уравнений Ax=b порядка n с разреженной матрицей A. Решить систему методом прогонки.
### на главной диагонали элементы равны 100, на 1, 2 и 3 наддиагоналях элементы равны 1, на 1 поддиагонали элементы равны 1.

In [125]:
n = 35
b = [i * np.exp(22. / i) * np.sin(9. / i) for i in range(1, n + 1)]

In [126]:
A = 100 * np.eye(n, n) + np.eye(n, n, k = 1) + np.eye(n, n, k = 2) + np.eye(n, n, k = 3) + np.eye(n, n, k = -1)

In [141]:
for j in range(10):
    print(j,":")
    print(A[-j][:])

0 :
[100.   1.   1.   1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.]
1 :
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   1. 100.]
2 :
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   1. 100.   1.]
3 :
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   1. 100.   1.   1.]
4 :
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   1. 100.   1.   1.   1.]
5 :
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.

In [142]:
b

[1477408851.871466,
 -117057.55359190803,
 647.9418744404106,
 761.5529359580186,
 396.6036774117512,
 234.13970794602747,
 155.64359568577365,
 112.91071868419407,
 87.27510669553946,
 70.6953593385805,
 59.32633548912332,
 51.16135921433577,
 45.075285725361724,
 40.3988403254057,
 36.713859586733946,
 33.748047845978,
 31.3177513072106,
 29.295305862967233,
 27.589615321560686,
 26.134173583293425,
 24.87943956037306,
 23.787845143344985,
 22.83044458656744,
 21.984614973516532,
 21.23244613953155,
 20.55959276972303,
 19.954442467040995,
 19.407503735775986,
 18.91094954833953,
 18.458272643929543,
 18.044022180858953,
 17.663600381907695,
 17.313103944456678,
 16.989199219471626,
 16.68902312463599]

Знаем, что в матрице А не более 5 ненулевых элементов => будем хранить для каждой строки только нееулевые элементы

In [157]:
nozero = []
nozero.append([100., 1., 1., 1.])
for i in range(1,n-3):
    nozero.append([1., 100., 1., 1., 1.])
nozero.append([1., 100., 1., 1.])
nozero.append([1., 100., 1.])
nozero.append([1., 100.])

In [158]:
nozero

[[100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0, 1.0],
 [1.0, 100.0, 1.0, 1.0],
 [1.0, 100.0, 1.0],


Сделала алгоритм для трехдиагональной матрицы В

In [172]:
B = 100 * np.eye(n, n) + np.eye(n, n, k = 1) + np.eye(n, n, k = -1)
B

array([[100.,   1.,   0., ...,   0.,   0.,   0.],
       [  1., 100.,   1., ...,   0.,   0.,   0.],
       [  0.,   1., 100., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ..., 100.,   1.,   0.],
       [  0.,   0.,   0., ...,   1., 100.,   1.],
       [  0.,   0.,   0., ...,   0.,   1., 100.]])

In [173]:
num = []
num.append([100., 1.])
for i in range(1,n-1):
    num.append([1., 100., 1.])
num.append([1., 100.])
num

[[100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0, 1.0],
 [1.0, 100.0]]

In [219]:
#алгоритм для трехдиагональной матрицы
def Thomas_algorithm(nozero,b,n):
    beta = np.zeros(n)
    alpha = np.zeros(n)
    gamma = np.zeros(n)
    
    #вычисление коэффицентов прогонки
    beta[0] = -b[0] / nozero[0][0]
    alpha[0] = - nozero[0][1]/nozero[0][0]
    gamma[0] = nozero[0][0]
    
    for i in range(1,n-1):
        gamma[i] = nozero[i][1] + nozero[i][0]*alpha[i-1]
        alpha[i] = - nozero[i][2] / gamma[i]
        beta[i] = (b[i] - nozero[i][0] * beta[i-1]) / gamma[i]

    gamma[n-1] = nozero[n-1][1] + nozero[n-1][0] * alpha[n-2]
    beta[n-1] = (b[n-1] - nozero[n-1][0] * beta[n-2]) / gamma[n-1]

    #вычесление x
    x = []
    x.append(beta[n-1])
    k = 0
    for i in range(n-2,-1,-1):
        x_k = alpha[i] * x[k] + beta[i]
        k += 1
        x.append(x_k)
        
    return(np.array(x))
    

#### Проверка

In [224]:
sol1 = np.linalg.solve(B,b)
sol1[-1::-1]

array([ 1.65224807e-01,  1.66542405e-01,  1.69733919e-01,  1.73169675e-01,
        1.76898927e-01,  1.80959765e-01,  1.85397250e-01,  1.90264803e-01,
        1.95626195e-01,  2.01558130e-01,  2.08153616e-01,  2.15526442e-01,
        2.23817169e-01,  2.33201276e-01,  2.43900402e-01,  2.56198108e-01,
        2.70462388e-01,  2.87178459e-01,  3.06997611e-01,  3.30811795e-01,
        3.59870759e-01,  3.95971903e-01,  4.41779313e-01,  5.01382563e-01,
        5.81323603e-01,  6.92592639e-01,  8.54771798e-01,  1.10533424e+00,
        1.52252260e+00,  2.28600166e+00,  4.01701981e+00, -7.38430491e+00,
        1.49596641e+03, -1.48941315e+05,  1.47755779e+07])

In [220]:
sol = Thomas_algorithm(nozero,b,n)

##### Решение алгоритмом прогонки

In [222]:
sol

array([ 1.65224807e-01,  1.66542405e-01,  1.69733919e-01,  1.73169675e-01,
        1.76898927e-01,  1.80959765e-01,  1.85397250e-01,  1.90264803e-01,
        1.95626195e-01,  2.01558130e-01,  2.08153616e-01,  2.15526442e-01,
        2.23817169e-01,  2.33201276e-01,  2.43900402e-01,  2.56198108e-01,
        2.70462388e-01,  2.87178459e-01,  3.06997611e-01,  3.30811795e-01,
        3.59870759e-01,  3.95971903e-01,  4.41779313e-01,  5.01382563e-01,
        5.81323603e-01,  6.92592639e-01,  8.54771795e-01,  1.10533454e+00,
        1.52249303e+00,  2.28895825e+00,  3.72139024e+00,  2.21756955e+01,
       -1.45973801e+03,  1.46599567e+05, -1.47755545e+07])